# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

### Step 1: Create Data Owners

In [1]:
!pip install syft

In [1]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

In [2]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

<VirtualWorker id:secure_worker #objects:0>

In [3]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)


### Step 2: Create the Model

In [4]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

### Step 3: Send a Copy of The Model to Alice and Bob

In [5]:
bobs_model = model.copy().send(bob) 
alices_model = model.copy().send(alice) 

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)


### Step 4: Train Bob's and Alice's Models (in parallel)

In [6]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).mean()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    print(f'Bob loss {bobs_loss}')

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).mean()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    print(f'alice loss {alices_loss}')


Bob loss 0.3030942380428314
alice loss 1.9678738117218018
Bob loss 0.179348424077034
alice loss 0.5936195850372314
Bob loss 0.11085241287946701
alice loss 0.18616658449172974
Bob loss 0.07254348695278168
alice loss 0.06474684923887253
Bob loss 0.0507582351565361
alice loss 0.028004225343465805
Bob loss 0.038045432418584824
alice loss 0.016377579420804977
Bob loss 0.0303395576775074
alice loss 0.01224452629685402
Bob loss 0.025420445948839188
alice loss 0.010387379676103592
Bob loss 0.022073887288570404
alice loss 0.009257806465029716
Bob loss 0.01963416486978531
alice loss 0.008393200114369392


### Step 5: Send Both Updated Models to a Secure Worker

In [7]:
alices_model.move(secure_worker) # the move function iterates on all objects in alice and call send on it. NOTEICE: inline function
bobs_model.move(secure_worker)


In [8]:
secure_worker._objects # can you identify those objects in the Secure worker?

{55418974901: Parameter containing:
 tensor([[ 0.6670, -0.1743]], requires_grad=True),
 10448677324: Parameter containing:
 tensor([0.4277], requires_grad=True),
 84106636451: Parameter containing:
 tensor([[ 0.0884, -0.2661]], requires_grad=True),
 16665178699: Parameter containing:
 tensor([0.1390], requires_grad=True)}

### Step 6: Average The Models

In [9]:
# A pytorch trick: to set the weights: you have to either access the data[idx] 
# or wrap the set function with (with torch.no_grad():)
# below we illustrate the first way
# the next cell demonstrates the complete way:

model.weight.data[0].set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
model.bias.data[0].set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

tensor([0.2833])

### Step 7: The complete solution over multiple training sessions

In [10]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0409) Alice:tensor(0.0180)
Bob:tensor(0.0227) Alice:tensor(0.0027)
Bob:tensor(0.0144) Alice:tensor(0.0007)
Bob:tensor(0.0095) Alice:tensor(0.0002)
Bob:tensor(0.0065) Alice:tensor(5.0212e-05)
Bob:tensor(0.0045) Alice:tensor(6.9123e-06)
Bob:tensor(0.0032) Alice:tensor(2.2666e-09)
Bob:tensor(0.0023) Alice:tensor(2.5128e-06)
Bob:tensor(0.0017) Alice:tensor(5.7918e-06)
Bob:tensor(0.0013) Alice:tensor(7.7244e-06)


In [11]:
preds = model(data)
loss = ((preds - target) ** 2).mean()

In [12]:
print(preds)
print(target)
print(loss.data)

tensor([[0.0963],
        [0.0762],
        [0.9045],
        [0.8845]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0094)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [13]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [14]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [15]:
bob_x_share *= 2
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [16]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted ( you are the dealer, thus still can see --and know everything-- but Alice and Bob did not really know anything about the overall deal --the sum of the numbers; the secrete--)

In this case, however, to keep the secrete is not about the relaiability / integrity of the person who holds it, rather the fact that those shareholders do not know each other!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant.

In [17]:
x = 5

Q = 9923 # the little fix; a very large prime number

bob_x_share = 1525 # give bob a random number.. try with negatives

alice_x_share = Q - bob_x_share + x

print(f'Bob share= {bob_x_share}')
print(f'Alice share= {alice_x_share}')
print(f'Add both shares= {bob_x_share + alice_x_share}... Wrong secrete')

Bob share= 1525
Alice share= 8403
Add both shares= 9928... Wrong secrete


In [18]:
# Decryption happens by summing all the shares together MODULUS some constant

(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so:  `x_share = (2,5,7)`

Even though normally those shares would be distributed among several workers, you can store them in ordered tuples like this for now.

In [2]:
import random

Q = 23740629843760239486723

# accepts a secrete and shares it among n_shahre, returns a tuple with n_share shares
def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

  # accepts a tuple of shares and return the decrypted values
def decrypt(shares):
    return sum(shares) % Q

In [20]:
shares = encrypt(x=7, n_share=10)
shares

(21413311274950407588844,
 1088603925299232792270,
 6099642636885892273100,
 16337439650190108821582,
 21961712137108077876233,
 15205740041382640925395,
 23588878004593626411185,
 21437261722122835770668,
 4998425966821140797257,
 10312763703207473663811)

In [21]:
decrypt(shares)

7

In [3]:
def add(a, b):
    c = list()
    assert(len(a) == len(b))
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [23]:
# run this block of code multiple times, what do you notice baout x and y?

x = encrypt(5)
print(x)
y = encrypt(7)
print(y)

z = add(x,y)
decrypt(z)

(7299532592613708800135, 12228675876276737611054, 4212421374869793075539)
(11544538949777722406489, 17633454543388988376052, 18303266194353768190912)


12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [54]:
BASE= 10  # integer numbers (normal numbers)
PRECISION= 4  # how many porecisions after the decimal point to encode
Q = 23740629843760239486723 

In [4]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [56]:
encode(-0.600)

23740629843760239345664

In [58]:
decode(9323)

0.9323

In [46]:
# run this cell multiple times and notice the changes

x = encrypt(encode(5.5))
y = encrypt(encode(5.5))
z = add(x,y)


print(f'Value of x = {x}')
print(f'Value of y = {y}')
print(f'Value of sum = {z}')
print(f'decoded sum = {decode(decrypt(z))}')

Value of x = (20987779365764538174036, 1953981087849393041759, 798869390146308276428)
Value of y = (22622800424900801229373, 19829344017331777214140, 5029115245287900535433)
Value of sum = (19869949946905099916686, 21783325105181170255899, 5827984635434208811861)
decoded sum = 11.0


# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [47]:
# Its a good idea to restrat the environment at this point

import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

<VirtualWorker id:secure_worker #objects:44>

In [48]:
x = th.tensor([1,2,3,4,5]) ##### the following steps work for integer values only

### Secret Sharing Using PySyft

We can share using the simple` .share()` method!

In [49]:
x = x.share(bob, alice, secure_worker) # implemets additive secrete shares
x # it's a pointer to three secrete shares

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:41676095587 -> bob:35039996605]
	-> (Wrapper)>[PointerTensor | me:75164760180 -> alice:26359791500]
	-> (Wrapper)>[PointerTensor | me:95923092632 -> secure_worker:7425582241]
	*crypto provider: me*

In [50]:
bob._objects # take a look at one of the shares.. Large random numbers --encrypted

{15375325955: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 34924558735: tensor([[0.],
         [0.]], requires_grad=True), 50740065398: tensor([[ 0.0335],
         [-0.0127]], grad_fn=<AddmmBackward>), 35039996605: tensor([1511731352967876930,  591925868599571724, 4210120092563609614,
          835677433872993700, 3223752319047256009])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [51]:
y = x + x
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:30063808948 -> bob:4371761319]
	-> (Wrapper)>[PointerTensor | me:73369911197 -> alice:70013394536]
	-> (Wrapper)>[PointerTensor | me:13756615944 -> secure_worker:75647072647]
	*crypto provider: me*

In [52]:
bob._objects # bob has another tensor.. why?

{15375325955: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 34924558735: tensor([[0.],
         [0.]], requires_grad=True), 50740065398: tensor([[ 0.0335],
         [-0.0127]], grad_fn=<AddmmBackward>), 35039996605: tensor([1511731352967876930,  591925868599571724, 4210120092563609614,
          835677433872993700, 3223752319047256009]), 4371761319: tensor([3023462705935753860, 1183851737199143448, 8420240185127219228,
         1671354867745987400, 6447504638094512018])}

In [53]:
y.get() # notice that we have doubled the values on each worker!

tensor([ 2,  4,  6,  8, 10])


### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using `.fix_precision()`
This is important when doing FL since the values you are sharing are decimal numbers.

**Q: what values are we sharing? what are the shared secretes?**

In [3]:
x = th.tensor([0.1, 0.2, 0.3, 0.4]) # a decimal tensor!

In [4]:
x

tensor([0.1000, 0.2000, 0.3000, 0.4000])

In [5]:
x = x.fix_precision() # call this function to encode the decimal tensor using fixed precision
x # notice that this is a tensor chain <-------

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400])

In [6]:
print(f'Type of x {type(x)}')
print(f'Type of x.child {type(x.child)}')
print(f'Type of x.child.child {type(x.child.child)}') # <-- this is how you get the data from a fixed precision tensor

Type of x <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>
Type of x.child <class 'syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor'>
Type of x.child.child <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>


In [22]:
# to decode the previous tensor
print(x.float_prec())

tensor([0.1000, 0.2000, 0.3000, 0.4000])


In [24]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([200, 400, 600, 800])

In [25]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000, 0.8000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [30]:
x = x.fix_prec().share(bob, alice)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:43948102131 -> bob:41549875499]
	-> (Wrapper)>[PointerTensor | me:98889542648 -> alice:96286304541]
	*crypto provider: me*

In [31]:
bob._objects

{1896941596: tensor([3337927225947607575, 4115017391059871510, 5758453651456544530]),
 19122572715: tensor([1981985574526026572, 2759075739638290507, 4402512000034963527]),
 21697835486: tensor([ 931215927921640947,  459606633707528827, 2077348708747142219,
         2518050637076306642, 2250413087966072035]),
 41549875499: tensor([3701295438721311877,  945318970846132605,   20024228591329413])}

In [0]:
y = x + x

In [33]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

# Final Project: Federated Learning with Encrypted Gradient Aggregation

Reuse your project from the Secuered Agggregator above to train the same model using the FL appraoch, but this time use the Additive Sharing Encryption so that the participating members have access to their own models only. There is no secure aggregator in this senario. 
Include four members in this project (Alice, Bob, Ted, and Carol). 

Hint: we will *share* the model with the shareholders, do addition on the encrypted models, and then aggregate the final model locally. 

In [1]:
!pip install syft

  Using cached https://files.pythonhosted.org/packages/d1/67/476640810609471e0f3a32c9f4388bf1318b773d0a64b116305d3b604dca/msgpack-0.6.1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/29/19/44753eab1fdb50770ac69605527e8859468f3c0fd7dc5a76dd9c4dbd7906/websocket_client-0.56.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4b/68/fe4806d3a0a5909d274367eb9b3b87262906c1515024f46c2443a36a0c82/Flask_SocketIO-4.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/75/14/fde90de4fc6722303ddbc183ec253c288dae8c7e4eb5a069ac49f14c9a0f/scikit_learn-0.21.2-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/64/b5/ebb1af4d843047ccd7292b92f5e5f8643153e8b95d14508d9fe3b35f7004/tblib-1.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/de/30/c241f360f769fd5a8623bf512de1b184a0473eaeaa6d32c7cda6cfeafef5/lz4-2.1.10-cp37-cp37m-win_amd64.whl
  Using cached https://files

In [1]:
# create cells as needed. Separate your solution into logical cells.

import syft as sy
import torch as th
from torch import nn, optim

In [2]:
hook = sy.TorchHook(th)

In [3]:
##bob.clear_objects
##alice.clear_objects


bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
ted = sy.VirtualWorker(hook, id="ted")
carol = sy.VirtualWorker(hook, id="carol")


bob.add_workers([alice, carol, ted])
alice.add_workers([bob, carol, ted])
carol.add_workers([alice, bob, ted])
ted.add_workers([alice, bob, carol])

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.],[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.],[0],[0],[1],[1.]], requires_grad=True)

data = data.fix_precision().share(bob, alice, ted, carol)
target = target.fix_precision().share(bob, alice, ted, carol)
# get pointers to training data on each worker by
# sending some training data to bob and alice
##bobs_data = data.send(bob)
##bobs_target = target.send(bob)

##alices_data = data.send(alice)
##alices_target = target.send(alice)

##teds_data = data.send(ted)
##teds_target = target.send(ted)

##carols_data = data.send(carol)
##carols_target = target.send(carol)

workers = ["bob", "alice", "ted", "carol"]

##model = nn.Linear(2,1)

#print(bob._objects)
#print(alice._objects)
#print(ted._objects)
#print(carol._objects)

In [4]:
model = nn.Linear(2,1)
#model = model.fix_precision()
#model = model.fix_precision().share(bob, alice, ted, carol)
optimizer = optim.SGD(params=model.parameters(), lr=0.1)

#print(model)
#print(bob._objects)
#print(alice._objects)
#print(ted._objects)
#print(carol._objects)
print(type(model))

<class 'torch.nn.modules.linear.Linear'>


In [5]:
iterations = 10
worker_iters = 5


for a_iter in range(iterations):

   ## bobs_opt = optim.SGD(params=model.parameters(), lr=0.1)
    print(a_iter)
    #optimizer = optim.SGD(params=model.parameters(), lr=0.1)
    
    model = model.fix_precision().share(bob, alice, ted, carol)
    
    for wi in range(worker_iters):
        # Train Bob's Model
        print(type(optimizer))
        optimizer.zero_grad()
        preds = model(data)
        bobs_loss = ((preds - target) ** 2).sum()
        bobs_loss.backward()

        optimizer.step()
    #bobs_loss = bobs_loss.get().data

       

    model = model.get().float_precision()


        
    
    ##print("Loss: " + bobs_loss)

0
<class 'torch.optim.sgd.SGD'>


AttributeError: 'FixedPrecisionTensor' object has no attribute 'attr'